# Example: subsampling from newspaper metadata stored in PostgreSQL

This notebook demonstrates how the `subsamplr` package can be used to subsample from records in a relational database containing metadata relating to historical newspaper articles.

The database includes information about the year of publication, the word count and the OCR quality of each article, and these variables are used as dimensions for representative subsampling.

We want our subsample to be representative in the sense that it preserves the joint distributions, along the three variables of interest, found in the wider dataset from which it is drawn. We also want to be able to condition on those variables, for instance by excluding newspaper articles published before a particular year.

## Install subsamplr and its dependencies

Before running this notebook you will need to install subsamplr and its dependencies. A convenient way to do this is to create a virtual environment, add that environment to Jupyter, and select it as the running kernel. Once this is done, we can import the required classes from subsamplr:

In [5]:
import os
import yaml
from io import StringIO
from numpy.random import seed as npseed  # type: ignore

os.chdir("..")
from subsamplr import BinCollection, DbUnitGenerator

## Set up some configuration parameters

The following YAML-formatted string contains configuration parameters for:
 - connecting to the database
 - specifying the subsampling variables of interest, their upper & lower bounds and bin sizes
 - querying the database to obtain data on each of the variables of interest.
 
**NOTE:** To run this example you will need to enter the database hostname (`db_host`) and your username (`db_user`) in the following configuration parameters. When querying the database (below) you will be prompted to enter your password.

In [2]:
config_str = """
    
    # Database connection parameters
    db_dialect: 'postgresql'
    db_host: '<HOSTNAME>'
    db_port: 5432
    db_user: '<USERNAME>'
    db_database: 'newspapers'
    
    # Subsampling dimensions
    variables:
        - {name: 'year', class: 'discrete', type: 'int', min: 1800,
            max: 1919, discretisation: 1, bin_size: 10}
        - {name: 'word_count', class: 'continuous', type: 'int',
            min: 0, max: 15000, bin_size: 1000}
        - {name: 'ocr_quality_mean', class: 'continuous', type: 'float',
            min: 0.6, max: 1, bin_size: 0.1}

    queries:
        article: |
            SELECT
                publication.fmp_id as nlp,
                publication.title as publication,
                publication.location as location,
                issue.issue_date as issue_date,
                CAST(EXTRACT(YEAR FROM issue.issue_date) AS INTEGER) as year,
                issue.input_sub_path as directory_path,
                article.word_count as word_count,
                article.ocr_quality_mean as ocr_quality_mean,
                article.title as article_title,
                article.fmp_id as article_fmp_id,
                issue.input_sub_path || '/' || article.fmp_id as article_id
            FROM
                publication,
                issue,
                article
            WHERE
                issue.publication_id=publication.id AND
                article.issue_id=issue.id;
    """

## Fetch data from the database

Records in the database will be used to generate subsampling "units", which in this case represent newspaper articles. We therefore construct a `DbUnitGenerator` object, and pass to it the configuration parameters for connecting to the database.

To reduce the time taken to execute the database query, we limit the number of results in this example to 50,000.

In [ ]:
# Read the YAML config.
config = yaml.safe_load(StringIO(config_str))

# Limit the number of results from the database query.
limit = 50000
query = config['queries']['article'].rstrip()[:-1] + f" LIMIT {limit};"

# Construct a database-driven subsampling unit generator.
ug = DbUnitGenerator(config['db_dialect'], config['db_host'],
                     config['db_port'], config['db_database'],
                     config['db_user'])

# Fetch the data.
df = ug.fetch_data(query)


## Each subsampling dimension appears as a column in the data table

The database query in the configuration is written such that each of the subsampling dimensions appears as a column in the fetched data.

Here the subsampling dimensions are `year`, `word_count` and `ocr_quality_mean`.

In [ ]:
df.head()

The data table must also contain a string identifier for each row. Here, the `article_id` serves this purpose.

## Generate subsampling units from the table rows and assign to bins

Each row in the data represents one article, and each of these is a unit in the sample from which we will draw a representative subsample.

The first step is to generate the units from the data, and assign them to bins according to their corresponding values along the three subsampling dimensions.

To do this we construct a `BinCollection` object, passing in the configuration parameters that specify the subsampling dimensions, their upper and lower bounds and their bin sizes.

We then pass the dimensions of the `BinCollection` to the `DbUnitGenerator`, so that it knows what information to extract from the data when converting it into a collection of subsampling units (articles).

Each unit consists of a pair of the form `(unit, values)`, where `unit` is a string identifier (the `article_id` in this case) and `values` is a tuple of corresponding values along each of the subsampling dimensions (`year`, `word_count` and `ocr_quality_mean`).

This collection of units is then assigned to bins in the `BinCollection`.

In [ ]:
# Generate newspaper article units from the data and assign to a BinCollection.
bc = BinCollection.construct(config, track_exclusions=True)

units = DbUnitGenerator.generate_units(
    df, unit_id="article_id", variables=bc.dimensions)

# Get all of the units from the generator.
generated_units = list(units)

for unit, values in generated_units:
    bc.assign_to_bin(unit, values)

After assignment, the `BinCollection` contains 34,803 units.

In [ ]:
bc.count_units()

This is fewer than the 50,000 rows in the data table. Why? Two reasons:

First, the lower and upper bounds of the dimensions in the `BinCollection` were specified in the config parameters:
```
# Subsampling dimensions
variables:
    - {name: 'year', class: 'discrete', type: 'int', min: 1800,
        max: 1919, discretisation: 1, bin_size: 10}
    - {name: 'word_count', class: 'continuous', type: 'int',
        min: 0, max: 15000, bin_size: 1000}
    - {name: 'ocr_quality_mean', class: 'continuous', type: 'float',
        min: 0.6, max: 1, bin_size: 0.1}
```
and any units whose values fall outside of those bounds are excluded from the collection.

Since we set `track_exclusions=True` when constructing the `BinCollection`, we have a record of those exclusions. There are 15,195 of them:

In [ ]:
bc.count_exclusions()

So we have 34,803 units (articles) in the `BinCollection` and 15,195 units in the data were excluded because they fell outside of the bounds of the collection. 

Together this accounts for 49,998 of the 50,000 rows in the data. What about the other 2 rows?

Those rows contained missing values and were therefore never turned into units at all:

In [ ]:
len(generated_units)

To summarise we have:
 - 50,000 rows of data
 - 49,998 rows without any missing data, from each of which a subsampling unit was generated
 - 34,803 units assigned to bins in the `BinCollection`
 - 15,195 units excluded from the `BinCollection` as their values are out of range of the configured dimensions.

## Extract a representative subsample

Now that we have a populated `BinCollection`, we can draw a subsample that is representative of the entire collection of units. 

This is achieved by using the (histogram) counts in each bin as weights for subsampling, and selecting units from each bin at random, according to the weight associated with that bin.

In [ ]:
# Construct a subsample of 5000 units.
k = 5000
seed = 147
npseed(seed)

sample = bc.select_units(k)

The subsample consists of a list of unit identifiers, which in this case identify particular newspaper articles by the combination of their:
 - seven digit title ID (a.k.a. NLP code)
 - four digit year of publication
 - four dight month and day of publication
 - article ID number within the newspaper issue.

In [ ]:
sample

## Summary

We have constructed a subsample of 5,000 articles, drawn from (a subset of) the 50,000 rows in our data, which is representative of that larger dataset in the sense that the joint distributions along all of the variables of interest (`year`, `word_count` and `ocr_quality_mean`) are preserved in the subsample.

Two of the rows of data contained missing values and were discarded. Another 15,195 rows fell outside the bounds of our collection of bins, as defined by our configuration parameters, and were ignored when drawing the sample.

The representative sample of size 5,000 was drawn from the remaining 34,803 articles whose values along the dimensions of interest fell within our prescribed bounds.

In [ ]:
len(sample)